In [1]:
#Solution BY Ashish Rana(1626670)

import numpy as np
from scipy.sparse.linalg import cg
from scipy.sparse import diags
import time


def conjugate_grad(Q, b, x=None):
    """
    Description
    ----------
    Solve a linear-quadratic problem min 1/2 x^TQx-b^Tx with conjugate gradient method with Q spd.
    Parameters
    ----------
    Q: 2d numpy.array of positive semi-definite (symmetric) matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Qx = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(Q, x) - b
    p = - r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Qp = np.dot(Q, p) # can be replaced by a function evaluation of Q times p
        alpha = r_k_norm / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - r
    return x


def prec_conjugate_grad(Q, W, b, x=None):
    """
    Description
    ----------
    Solve a linear-quadratic problem min 1/2 x^TQx-b^Tx with preconditioned conjugate gradient method with Q spd.
    Parameters
    ----------
    Q: 2d numpy.array of positive semi-definite (symmetric) matrix
    W: 2d numpy.array preconditioning matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Qx = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(Q, x) - b
    z = np.dot(W, r)
    p = - z
    r_k_z = np.dot(r, z)
    for i in range(2*n):
        Qp = np.dot(Q, p)
        alpha = r_k_z / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        z = np.dot(W, r)
        r_k_z_plus1 = np.dot(r, z)
        beta = r_k_z_plus1 / r_k_z
        r_k_z = r_k_z_plus1
        if np.dot(r, r) < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - z
    return x



#Task 1
print("start for 1st Q")
n =[10,50,250,1250]
for n in n:
    q=np.diag((np.arange(n)+1))
    Q=2*q
    W=np.linalg.inv(np.diag(np.diag(q))) 
    b = np.ones(n)
    t1=time.time()
    x1=conjugate_grad(Q,b)
    t2=time.time()
    print (t2 - t1)
    t3=time.time()
    x2=prec_conjugate_grad(Q,W,b)
    t4=time.time()
    print(t4-t3)
    
 

print("\nStart for 2nd Q")
n =[10,50,250,1250]
for n in n:
    a=1/(n-1)**2
    q=diags([-a,2*a,-a],[-1,0,1],shape=(n,n)).toarray()
    Q=2*q
    W=np.linalg.inv(np.diag(np.diag(q))) 
    b = np.ones(n)
    t1=time.time()         
    x1=conjugate_grad(Q,b)
    t2=time.time()
    print (t2 - t1)
    t3=time.time()        
    x2=prec_conjugate_grad(Q,W,b)
    t4 = time.time()
    print (t4 - t3)          


start for 1st Q
Itr: 9
0.0012922286987304688
Itr: 0
0.0012216567993164062
Itr: 33
0.0017132759094238281
Itr: 0
0.00052642822265625
Itr: 79
0.015502214431762695
Itr: 0
0.0010044574737548828
Itr: 183
1.390531301498413
Itr: 0
0.019780635833740234

Start for 2nd Q
Itr: 4
0.0006811618804931641
Itr: 4
0.0005185604095458984
Itr: 24
0.0012125968933105469
Itr: 24
0.0011620521545410156
Itr: 124
0.00583648681640625
Itr: 124
0.0078012943267822266
Itr: 624
0.19283723831176758
Itr: 624
0.4191431999206543


In [2]:
#Task 2
def MatrixQ_vector_mul(x):
    n=np.size(x)
    y=np.empty(n)
    for i in range(n):
        if(i==0):
            y[i]=x[i]-x[i+1]
            
        elif(i==(n-1)):
            y[i]=x[i]-x[i-1]
            
        else:
            y[i]=2*x[i]-x[i-1]-x[i+1]
        
    return 2*y

b=np.zeros(50000)
for i in range(len(b)):
    if i%2==0:
        b[i]=1
    else:
        b[i]=-1

            
def conjugate_grad_specific(b, x=None):
    
    n = len(b)
    if not x:
        x = np.ones(n)
    r =  MatrixQ_vector_mul(x)- b
    p = - r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Qp = MatrixQ_vector_mul(p) 
        alpha = r_k_norm / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - r
    return x    

conjugate_grad_specific(b)

#The special feauture about the system matrix Q in this case is that it is not invertible since 
#the determinant of Q is 0.
# Due to this there is not a unique solution possible for Qx=b or x=Q^-1*b, because 
#Q inverse doesn't exist.
# But Q is still a symmetric and positive semi-definite matrix which implies that we are solving a 
# convex optimization problem(though not strictly convex), and that's why we are still getting
# a solution.

Itr: 24999


array([ 6251. ,  6250.5,  6250.5, ..., -6248.5, -6248.5, -6249. ])

In [3]:
#Task 3
from scipy import optimize
def test_nlcg(nlcg):
    print("\n# Test nlcg on Rosenbrock function.")
    def frosenbrock(z):
        x,y = z
        return (1-x)**2 + 100*(y- x**2)**2

    x0=np.zeros(2)

    sol_nlcg = nlcg(frosenbrock,x0)
    print("Test (should be 0):\n", frosenbrock(sol_nlcg))

def pdrv(f, x, i, h=1e-6):
    
    step = np.zeros(np.size(x))
    step[i] = h
    return (f(x+step) - f(x-step))/(2.*h)

def grad(f, x, h=1e-6):
    
    n = np.size(x)
    grad = np.zeros(n)
    
    for i in range(n):
        grad[i] = pdrv(f, x, i, h)

    return grad

def nlcg(f, x, tol=1e-9, max_it = 10000):
    r = -grad(f,x)
    p = r
    r_k_norm = np.dot(r,r)
    for i in range(max_it):
        def k(y):
            return  np.dot(grad(f,x+y*p),p)
        sol=optimize.root_scalar(k,bracket=[0, 3], method='brentq')
        alpha = sol.root
        x += alpha * p
        r_prev=r
        r = -grad(f,x)
        r_kplus1_norm=np.dot(r,r-r_prev)
        beta = max(r_kplus1_norm/r_k_norm,1)
        r_k_norm = np.dot(r,r)
        if r_kplus1_norm < tol:
            print ('Itr:', i)
            break
        p = beta * p + r
    return x

test_nlcg(nlcg)    


# Test nlcg on Rosenbrock function.
Itr: 10
Test (should be 0):
 0.00028398218971110035
